In [ ]:

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)# 把資料路徑mapping到google drive

!pip install line-bot-sdk flask flask-ngrok  # 需重新安裝套件
!pip install pandas imgurpython SpeechRecognition pydub filetype
!pip install opencc-python-reimplemented

In [58]:
######################### import ##########################
#  1. 引用flask Web Server套件
from flask import Flask, request, abort, jsonify
#  1. 載入json處理套件
import json
#  1. 外部連結自動生成套件
from flask_ngrok import run_with_ngrok
#  1. 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 
from linebot import LineBotApi, WebhookHandler
#  1. 引用 InvalidSignatureError(無效簽章錯誤)
from linebot.exceptions import InvalidSignatureError
#  1. 引入Follow事件
from linebot.models.events import (
    FollowEvent,
    UnfollowEvent
)
#  1. 引入各式事件
from linebot.models import (
    MessageEvent, 
    PostbackEvent,
)
#  1. 引入各種Message
from linebot.models import (
    # 傳入消息
    TextMessage, ImageMessage, LocationMessage, VideoMessage, AudioMessage,
    # 送出消息
    TextSendMessage, ImageSendMessage, AudioSendMessage, VideoSendMessage,
    # 模板(送出)消息
    TemplateSendMessage,
    # 輪播消息
    CarouselTemplate,CarouselColumn,
    ImageCarouselTemplate,ImageCarouselColumn
)
#  1. 引入各種Action
from linebot.models import (
    # 傳入消息
    MessageAction, 
    PostbackAction,
    URIAction,
    DatetimePickerAction,
    LocationAction,
    CameraAction, 
    CameraRollAction, 
    PostbackTemplateAction, 
    MessageTemplateAction,
    URITemplateAction,
)
#  1. 引入其他訊息元件
from linebot.models import (
    # 快捷回應、快捷按鈕
    QuickReply, QuickReplyButton,
    # 將bubble類型的json 進行轉換變成 Python可理解之類型物件
    # 將該物件封裝進 Flex Message中
    FlexSendMessage,BubbleContainer
)
#  1. 引入訊息寄件者
from linebot.models import (
    Sender,
)
#  1. 引入按鍵模板
from linebot.models.template import (
    ButtonsTemplate,
)
#  1. 引入Menu選單
from linebot.models import (
    RichMenu, RichMenuSize, RichMenuArea, RichMenuBounds,
)
#  1. 引入有的沒的
import csv, pandas as pd, random, os
from imgurpython import ImgurClient
import imghdr 
import speech_recognition as sr
import requests
from pydub import AudioSegment
from opencc import OpenCC
#  沒時間換成串資料庫了OTZ

In [59]:
######################### globals #########################
#  2. *是否在colab上運作
onColab=True
#  2. *app名
appname="選擇困難偵探事務所"
#  2. *存放資訊的各檔名
userlog="app-users.log"     # 用戶資訊
eventlog="app-event.log"    # 事件資訊
bugreportlog="app-bugreport.log"# bug回報資訊
useroperatestatecsv="user_operate_state.csv"# 用戶操作狀態
#  2. *存放檔案的各資料夾名
sys_files="sys_files"     # 系統圖片資料夾
store_imgs="store_imgs"     # 回傳圖片資料夾
store_videos="store_videos" # 回傳影片資料夾
store_audios="store_audios" # 回傳音訊資料夾
#  2. *Channel驗證內容，請使用自己的Channel驗證內容
Channel_Access_token="VbuxTaJPqTwUBTk11k7GFpY83s1f+KI0KWbmhnyzBUFXwLP/5sIJXCdWaBA6etI1W0xIPpkB1NCTcJhQiRCvtyIngJGZJZgWEQib3ECHx88Gm8CO8taUZYiQxwgPa97FfeBHOPAB2w5sRJEE5LD7cwdB04t89/1O/w1cDnyilFU="
Channel_Secret="239d23fd3fece5abe9df334610b7a863"
#  2. * imgur API 驗證內容，請使用自己的驗證內容
imgur_ClientID="416808f931aeadb"
imgur_ClientSecret="aa2cfeca6ec540264389670d4f9dd839cbf3fbeb"
imgur_Client= ImgurClient(imgur_ClientID,imgur_ClientSecret)
#-------------------------- path --------------------------
#  2. 在colabg上起始路徑就是Google Drive，不然就是空字串
startpath="/content/drive/MyDrive" if onColab else ""
#  3. app根路徑
apppath_root=f"{startpath}/forLineChatBot_{appname}"
#  4. app存放資訊路徑
apppath_userlog=f"{apppath_root}/{userlog}"
apppath_eventlog=f"{apppath_root}/{eventlog}"
apppath_bugreportlog=f"{apppath_root}/{bugreportlog}"
apppath_useroperatestatecsv=f"{apppath_root}/{useroperatestatecsv}"
#  4. app存檔資料夾路徑
apppath_sysfiles=f"{apppath_root}/{sys_files}"
apppath_imgs=f"{apppath_root}/{store_imgs}"
apppath_videos=f"{apppath_root}/{store_videos}"
apppath_audios=f"{apppath_root}/{store_audios}"
#  4. 圖片url
imgurl_office='https://i.imgur.com/5hFBVFK.png'
imgurl_mainActor='https://i.imgur.com/qgNQUNV.jpeg'
imgurl_fortuneTeller='https://i.imgur.com/fb0wKSQ.jpeg'
imgurl_CARD='https://i.imgur.com/ppAOLMS.png'
imgurl_cardFront='https://i.imgur.com/xdLWbPS.png'
imgurl_cardBack='https://i.imgur.com/UoaD0Cw.png'
imgurl_Bang='https://i.imgur.com/5rMUSz0.gif'
imgurl_Miss='https://i.imgur.com/s68xxMM.gif'
OutdoorRMpic=f'{apppath_sysfiles}/outdoorRichMenu.png'
OfficeRMpic=f'{apppath_sysfiles}/officeRichMenu.png'
diceRMpic=f'{apppath_sysfiles}/officeRichMenu_Dice.png'
ftRMpic=f'{apppath_sysfiles}/ftRichMenu.png'


In [60]:
from google.cloud import storage
storage_client = storage.Client.from_service_account_json(f'{apppath_sysfiles}/service_account.json')

In [61]:
  #塔羅
TarotCards=[
    "愚者　The Fool",
    "魔術師　The Magician",
    "女祭司　The High Priestess",
    "皇后　The Empress",
    "皇帝　The Emperor",
    "教皇　The Hierophant",
    "戀人　The Lovers",
    "戰車　The Chariot",
    "力量　The Strength",
    "隱者　The Hermit",
    "命運之輪　Wheel of Fortune",	
    "正義　Justice",
    "倒吊男　The Hanged Man",
    "死神　Death",
    "節制　Temperance",
    "惡魔　The Devil",
    "塔　The Tower",
    "星星　The Star",
    "月亮　The Moon",
    "太陽　The Sun",
    "審判　Judgement",
    "世界　The World"
]
TarotDetails={
    "愚者　The Fool":("https://i.imgur.com/lQbFLgf.png","流浪 (潛在的、可能性、新開始、天真單純)","此牌在傳統塔羅牌中是代表“愚人”，即大智若愚的純真之人。這張牌預示著要是我們能有一顆相信夢想的純真之心，那么，我們就能貫徹夢想，向前邁進。因此，這張牌也含有解決問題、開導迷惘者的含義。"),
    "魔術師　The Magician":("https://i.imgur.com/SZ99ASd.png","創造 (知識、力量、控制力)","這張牌隱喻著你天生擁有創造能力和暗示著你的潛在能力。不過，若想發揮這一能力，則需要堅強的意志與正確的目的。要是使用不正當的話，也可能招來可怕的後果！！"),
    "女祭司　The High Priestess":("https://i.imgur.com/d4wwQGf.png","智慧 (隱匿的影響、神秘的智慧)","這個世界上，所有的東西都能分為善與惡、明與暗等相對的兩部分。女祭司就是憑藉著理性的鍛鍊，來分辨善惡、明暗，甚至就連罪惡及黑暗，也能由她給予明確地指示。女祭司這張牌象徵的是一種無形的，隱藏在靈魂深處、自然被誘發出來的情感，同時也表示女性的直覺力和知識的權威。"),
    "皇后　The Empress":("https://i.imgur.com/j9arUp8.png","豐收 (創造力、生長、生產)","皇后代表著洋溢著母性溫柔的女性特質。母性所隱喻的是生命的誕生、孕育、大地的豐收及物質的繁榮。她象徵著幸福的愛情、優厚的物質與滿足的生活。相對地，她也提醒人們，過於安逸的生活會導致奢侈、懈怠們不可以鬆動努力與上進心，否則容易掉入墮落的深淵。"),
    "皇帝　The Emperor":("https://i.imgur.com/3nUEqb4.png","支配 (領導權、自我控制)","皇帝，是男性象徵的代表，亦具有父親深沉肅穆的形象。從它的牌號IV看來，它掌管著安定與力量，暗示著有主管物質社會的支配力。皇帝象徵著男性的意志與熱情，並且充滿了勇氣與行動力，他是絕對的統治者。相反地，在擁有權力和地位的背後，也反映出自以為是的另一面，有著高處不勝寒的隱喻在內。"),
    "教皇　The Hierophant":("https://i.imgur.com/wmFh69H.png","援助 (學問、整合、表達)","和四號牌皇帝所代表物質社會的權威相比，祭司所代表的，則是精神上的權威。在宗教上，祭司象徵著理髮與道德、深沉的慈悲心和洞察力。他為人們解困，並用言論指引他們走向正途，所以這是一張代表著呼喚人類良心與善意覺醒之牌。此外，它也象徵著保守的道德與傳統的知識，一旦你忽視了它，就可能會招致危險。"),
    "戀人　The Lovers":("https://i.imgur.com/JYvEFJe.png","結合 (吸引、誘惑)","戀人這張牌象徵兩性關係中充滿磁性的吸引力，彼此深愛著對方，沉浸在甜蜜與幸福中。相對地，它也暗示了戀愛中出現的迷惘，如：自由被對方所限制、第三者的出現等等。因此，戀愛是既美麗又殘酷的。"),
    "戰車　The Chariot":("https://i.imgur.com/uDDV7U0.png","勝利 (成功、變化、平衡)","這張牌預示著變化、運動和旅行。牌上展示的精神能力的提升、個人的進步以及力量都代表你能夠同時做很多事情。人類一般都擁有野心與競爭的本能、上進心與克己心的意志力，兩者平衡互存就會在人生的道路上取得勝利。若一旦失衡，力量就反而會轉變成為反噬的野獸。"),
    "力量　The Strength":("https://i.imgur.com/NHhdoHC.png","意志 (發現內在的力量、面對逆境的勇氣)","人類對於自己所需面對的難題，有著無所畏懼、勇往直前的勇氣，這就是該牌的意義。它亦象徵著堅強的意志力，若欠缺了這此，那么人類就無法克服困難，向前邁進了。"),
    "隱者　The Hermit":("https://i.imgur.com/E9JMWz6.png","尋求 (完成、沉思、洞察力)","冥冥中神明的智慧將降臨到你的頭上。體驗人生的多種經歷，學會如何面對自己，消除自大、占有欲和其他消極情感。總之，此牌亦有自我探索的含意，向過去的自己作告別，反樸歸真，尋找內在的自我。"),
    "命運之輪　Wheel of Fortune":("https://i.imgur.com/Drp9v0o.png","輪迴 (變化、事情的轉折)","源自佛教的術語“輪迴”。生命的表現形式一直都是周而復始、生生不息。種下怎么樣的因，便會結出怎么樣的果。這張牌亦有“宿命”的感覺，只要求問者留心機遇，實踐積極的態度，這樣就能影響生命之輪的運轉，從而獲得好運，正所謂“風水輪流轉”。"),	
    "正義　Justice":("https://i.imgur.com/QrFzoI3.png","均衡 (平衡、性質相反、清澈)","在與大自然的相親相愛中思索自我的真實本性。要使自身的精力、能量變得平衡、和諧。當你有所改變，就會取得好的結果。另外，這張牌也警示我們，不論在何時何地，都必須堅守道德和正義，並且還要有寬容的心胸，不可有絲毫的懈怠。"),
    "倒吊男　The Hanged Man":("https://i.imgur.com/62sXMNJ.png","犧牲 (等待、預言的力量)","倒吊男之牌代表反求諸己、自我檢討與反省。它有點象放射性治療或瑜珈療法，用逆性思考的方式解決心理上的障礙。亦顯示只要我們耐心等待，噩運仍會如肉體一般，消失殆盡。“犧牲”是誠實地付出自己，去換取幸福。"),
    "死神　Death":("https://i.imgur.com/Yxq5tFr.png","結束 (轉變、解放、更新)","這張牌提示的是轉變，持久的變化，思想意識的變遷、更新。這張牌同時也預示著——無論是什麼事情的結束，在我們迎接結果來臨之時，同時也是往新的人生與目標展開新的旅程。最重要的是我們要把眼光看向未來。"),
    "節制　Temperance":("https://i.imgur.com/SHXgZxB.png","淨化 (自我控制、和諧一致、克己自製)","自我控制力將有助你與他人和睦相處，共同完成某種任務。只要你獲得精神上、欲望上的平衡，你就能夠成功，得到你未曾得到的東西。這張牌教導我們的是：人生在面臨心中各種各樣的欲望時，最重要的是如何將它控制得宜。"),
    "惡魔　The Devil":("https://i.imgur.com/hF25HaS.png","誘惑 (共鳴、可變通性、決定)","致命的吸引力總是帶來欲望的糾纏！糾纏的邪惡特徵就是忽略了真愛的本質，釀成悲慘的結果。凡事不要只看錶象，運用自己的智慧，堅持信念，不可輕易放棄，這樣希望就會出現。"),
    "塔　The Tower":("https://i.imgur.com/JjmZDF4.png","毀滅 (強行的變化、一個循環的終結)","這張牌預示著一個生命中的周期正在開始或者終結，你正面臨著變化與衝突。你要細心謹慎、隨機應變，否則將會一片混亂。這張牌亦暗示著破滅的危機，它警告我們，不可大過驕傲自大，應該謙虛內斂，謹守本分才是。"),
    "星星　The Star":("https://i.imgur.com/d8dUNtd.png","希望 (照明、靈感、超自然能量)","星這張牌意味著前衛的思想，新的知識、希望與理想，預示著諾言的兌現，靈感與謹慎之愛。星所要傳達的，就是希望我們能夠不吝惜地盡心付出我們的真愛，藉此來培育希望，邁向光明的未來。"),
    "月亮　The Moon":("https://i.imgur.com/TjdsLQH.png","不安 (超自然能力、夢境、欺騙)","月亮連線著想像、聯想的精神世界，因為它有陰晴圓缺，所以又代表著“變化”。這張牌所透露的訊息，是要我們注意潛意識的活動，及早發現事情的曖昧性及不安定感。所以即使身處再穩定的狀態之下，也絕不可鬆懈倦怠，憑直覺找出隱匿的敵人。"),
    "太陽　The Sun":("https://i.imgur.com/hGTPYyi.png","生命 (創造力、共同工作、成就)","太陽的光芒，照耀萬物，賜予無盡的生命活力。這張牌所要表達的是“生命的喜悅”，同時它也教導我們所有生命的喜悅，皆來自健康的養育與繁榮的生活。它還展示了人類的團體精神，合作精神與非凡的創造力。通過這張牌，你會找到通往真理的路途。"),
    "審判　Judgement":("https://i.imgur.com/2z0tp8a.png","復活 (更新、覺醒)","審判這張牌告訴我們，要發現真相，接受新的價值觀，如此一來才能擁有新的人生。它暗喻的是心靈的更新與再生。"),
    "世界　The World":("https://i.imgur.com/vEkHkXA.png","達成 (充滿、成功、勝利、宇宙的意識)","一個循環終結了，很快便會取得成功，獲得回報了！你的精神將會升華至更高的層次。世界這張牌強調造物主調和陰陽的能力，只要你擁有感恩的心，就必定能在自己創造的世界中，尋求到幸福與快樂。")
}
#------------------------- acting -------------------------
#  4. 主角發送人-泫則睏男
senderMainActor=Sender(
    name="名偵探-泫則睏男",
    icon_url=imgurl_mainActor
)
#  4. 配角發送人-占卜師
senderFortuneTeller=Sender(
    name="占卜師",
    icon_url=imgurl_fortuneTeller
)
#  4. 起始訊息
startframe_Messages=[
    TemplateSendMessage(
        alt_text='Image Carousel Template',
        template=ImageCarouselTemplate(
            columns=
            [
                ImageCarouselColumn(
                    image_url=imgurl_office,
                    action=PostbackTemplateAction(
                        label='"泫則偵探事務所"',
                        text=None,
                        data='None'
                    )
                )
            ]
        )
    ),
]
firstask_Messages=[
    TemplateSendMessage(
        alt_text='Image Carousel Template',
        template=ImageCarouselTemplate(
            columns=
            [
                ImageCarouselColumn(
                    image_url=imgurl_mainActor,
                    action=PostbackTemplateAction(
                        label='"泫則睏男(32)"',
                        text=None,
                        data='None'
                    )
                )
            ]
        )
    ),
    TextSendMessage(
        sender=senderMainActor,
        text='請問有什麼事情嗎？'
    ),
]
#  4. 占卜攤訊息
ftask_Messages=[
    TemplateSendMessage(
        alt_text='Image Carousel Template',
        template=ImageCarouselTemplate(
            columns=
            [
                ImageCarouselColumn(
                    image_url=imgurl_fortuneTeller,
                    action=PostbackTemplateAction(
                        label='"占卜師(??)"',
                        text=None,
                        data='None'
                    )
                )
            ]
        )
    ),
    TextSendMessage(
        sender=senderFortuneTeller,
        text='讓我揭示你的命運...'
    ),
]

In [62]:
#------------------------ messages ------------------------
#  5. 名片輪播圖文訊息
def bussinesscardICTM():
    image_carousel_template_message = TemplateSendMessage(
        alt_text='Image Carousel Template',
        template=ImageCarouselTemplate(
            columns=
            [
                ImageCarouselColumn(
                    image_url=imgurl_cardFront,
                    action=PostbackTemplateAction(
                        label='名片的正面',
                        text='名片正面：\n"選擇困難偵探事務所\n- 為您解決選擇上的困難"',
                        data='None'
                    )
                ),
                ImageCarouselColumn(
                    image_url=imgurl_cardBack,
                    action=PostbackTemplateAction(
                        label='名片的背面',
                        text='名片背面：\n"名偵探 泫則睏男"\n還有一些聯絡資訊',
                        data='None'
                    )
                ),
                ImageCarouselColumn(
                    image_url=imgurl_CARD,
                    action=PostbackTemplateAction(
                        label='名片上的地址有些熟悉',
                        text='那地方就在常去的咖啡廳樓上，去看看吧。',
                        data='goSTART'
                    )
                )
            ]
        ),
    )
    return image_carousel_template_message
#  5. 開場輪播圖文訊息
def openingICTM():
    image_carousel_template_message = TemplateSendMessage(
        alt_text='Image Carousel Template',
        template=ImageCarouselTemplate(
            columns=
            [
                ImageCarouselColumn(
                    image_url='https://i.imgur.com/9VvPoqX.jpg',
                    action=PostbackTemplateAction(
                        label='這一天...',
                        text=None,
                        data='opening'
                    )
                )
            ]
        )
    )
    return image_carousel_template_message
#  5. 塔羅選項輪播模板訊息
def tarotCTM(cardsdetails_,intN):
    '''
    options ex:
        {
            "塔羅1":(img_url,meaning,detail),
            ......
        }
    '''
    items_=[]
    items_.append(
        QuickReplyButton(
            action=PostbackTemplateAction(
                label='重抽',
                data=f'functionTarot&{intN}&Ready'
            )
        )
    )
    items_.append(
        QuickReplyButton(
            action=PostbackTemplateAction(
                label='結束',
                data='fortuneTeller'
            )
        )
    )
    columns_=[]
    for card_ in cardsdetails_:
        columns_.append(
            CarouselColumn(
                thumbnail_image_url=cardsdetails_[card_][0],
                title=card_,
                text=cardsdetails_[card_][1],
                actions=[
                    PostbackTemplateAction(
                        label='詳細',
                        data=f'FTspeak&【{card_}】\n{cardsdetails_[card_][2]}'
                    ),
                ]
            )
        )
    carousel_template_message = TemplateSendMessage(
        sender=senderFortuneTeller,
        alt_text='Carousel Template',
        template=CarouselTemplate(
            columns=columns_,
        ),
        quick_reply=QuickReply(
            items = items_
        )
    )
    return carousel_template_message
def do_FTspeak(event,others_):
    line_bot_api.reply_message(
        event.reply_token,
        [
            TextSendMessage(
                sender=senderFortuneTeller,
                text=others_[0]
            )
        ]
    )
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.link_rich_menu_to_user(event.source.user_id,ftRMid)

In [63]:
#-------------------------- Menu --------------------------
#  5. 將圖文選單配圖
def setRichMenu(richMenu,imgpath_):
    result_ = None
    try:
        richMenuId_ = line_bot_api.create_rich_menu(rich_menu=richMenu)
        with open(imgpath_, 'rb') as f:
            line_bot_api.set_rich_menu_image(richMenuId_,"image/jpeg",f)
        result_ = True
    except Exception:
        result_ = False
    return richMenuId_,result_
#  5. 門口圖文選單
def outdoorRichMenu():
    rich_menu_to_create=RichMenu(
        size = RichMenuSize(width=800, height=270),
        selected = True,
        name = 'outdoorRichMenu',
        chat_bar_text = '[偵探事務所外] 選項',
        areas=[
            RichMenuArea(
                bounds=RichMenuBounds(x=0, y=0, width=220, height=270),
                action=PostbackTemplateAction(
                    label='看看佈告',
                    text=None,
                    data='lookPosts'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=220, y=0, width=220, height=270),
                action=PostbackTemplateAction(
                    label='占卜攤',
                    text=None,
                    data='fortuneTeller'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=440, y=0, width=360, height=270),
                action=PostbackTemplateAction(
                    label='往樓上事務所',
                    text=None,
                    data='walkinOffice'
                )
            )
        ]
    )
    return rich_menu_to_create
#  5. 事務所圖文選單
def officeRichMenu():
    rich_menu_to_create=RichMenu(
        size = RichMenuSize(width=800, height=270),
        selected = True,
        name = 'officeRichMenu',
        chat_bar_text = '[泫則偵探事務所] 選項',
        areas=[
            RichMenuArea(
                bounds=RichMenuBounds(x=0, y=0, width=660, height=90),
                action=PostbackTemplateAction(
                    label='聽聽介紹',
                    text=None,
                    data='introduction'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=0, y=90, width=220, height=180),
                action=PostbackTemplateAction(
                    label='電腦選購',
                    text=None,
                    data='functionBuyComputer'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=220, y=90, width=220, height=180),
                action=PostbackTemplateAction(
                    label='送禮建議',
                    text=None,
                    data='functionGiftAdv'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=440, y=90, width=220, height=180),
                action=PostbackTemplateAction(
                    label='宇宙機率空間取樣法',
                    text=None,
                    data='functionDice'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=660, y=0, width=140, height=270),
                action=PostbackTemplateAction(
                    label='離開事務所',
                    text=None,
                    data='walkoutOffice'
                )
            )
        ]
    )
    return rich_menu_to_create
#  5. 骰子圖文選單
def diceRichMenu():
    rich_menu_to_create=RichMenu(
        size = RichMenuSize(width=800, height=270),
        selected = True,
        chat_bar_text = '[宇宙機率空間取樣法] 選項',
        name = 'diceRichMenu',
        areas=[
            RichMenuArea(
                bounds=RichMenuBounds(x=0, y=0, width=660, height=90),
                action=PostbackTemplateAction(
                    label='骰子介紹',
                    text=None,
                    data='introduction_Dice'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=0, y=90, width=220, height=180),
                action=PostbackTemplateAction(
                    label='自訂骰子一覽',
                    text=None,
                    data='functionDiceList'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=220, y=90, width=220, height=180),
                action=PostbackTemplateAction(
                    label='俄羅斯輪盤骰',
                    text=None,
                    data='functionBasketBet'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=440, y=90, width=220, height=180),
                action=PostbackTemplateAction(
                    label='經典亂數骰',
                    text=None,
                    data='functionNormalDice'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=660, y=0, width=140, height=270),
                action=PostbackTemplateAction(
                    label='回事務所主選單',
                    text=None,
                    data='walkinOffice'
                )
            )
        ]
    )
    return rich_menu_to_create
#  5. 占卜攤圖文選單
def ftRichMenu():
    rich_menu_to_create=RichMenu(
        size = RichMenuSize(width=800, height=270),
        selected = True,
        name = 'ftRichMenu',
        chat_bar_text = '[占卜攤] 選項',
        areas=[
            RichMenuArea(
                bounds=RichMenuBounds(x=0, y=0, width=660, height=90),
                action=PostbackTemplateAction(
                    label='占卜介紹',
                    text=None,
                    data='introduction_FT'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=0, y=90, width=330, height=180),
                action=PostbackTemplateAction(
                    label='五連抽',
                    text=None,
                    data='functionTarot&5'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=331, y=90, width=330, height=180),
                action=PostbackTemplateAction(
                    label='單抽',
                    text=None,
                    data='functionTarot&1'
                )
            ),
            RichMenuArea(
                bounds=RichMenuBounds(x=661, y=0, width=140, height=270),
                action=PostbackTemplateAction(
                    label='離開占卜攤',
                    text=None,
                    data='walkoutOffice'
                )
            )
        ]
    )
    return rich_menu_to_create

    

In [64]:
#------------------- QuickReplyButtons --------------------
#  5. 範例-地理位置快捷按鈕：跳出地理位置
def QRB_Location(label_="地理位置",img_url=None):
    locationQRB = QuickReplyButton(
        action=LocationAction(label=label_),
        image_url=img_url
    )
    return locationQRB
#----------------------- QuickReply -----------------------
#  6. 範例-快捷回應：將QuickReplyButton(s)加入QuickReply的items
def produceQuickReplyItem(*QuickReplyButtons):
    quickReplyList = QuickReply(
        items = QuickReplyButtons
    )
    return quickReplyList

In [65]:
######################### prepare #########################
#  9. 設定Server啟用細節
app = Flask(
    __name__,
    static_url_path = f"/{appname}" , 
    static_folder = f"./{appname}/"
)
run_with_ngrok(app)
#  9. 生成實體物件
line_bot_api = LineBotApi(Channel_Access_token)
handler = WebhookHandler(Channel_Secret)



In [66]:
######################## RichMenu #########################
list_=line_bot_api.get_rich_menu_list()
for RM_ in list_:
    print("Deleting RM_：",RM_.rich_menu_id)
    line_bot_api.delete_rich_menu(RM_.rich_menu_id)
list_=line_bot_api.get_rich_menu_list()
print(list_)

# 門口圖文選單
OutdoorRMid,_=setRichMenu(outdoorRichMenu(),OutdoorRMpic)
# 事務所圖文選單
OfficeRMid,_=setRichMenu(officeRichMenu(),OfficeRMpic)
# 骰子圖文選單
diceRMid,_=setRichMenu(diceRichMenu(),diceRMpic)
# 占卜圖文選單
ftRMid,_=setRichMenu(ftRichMenu(),ftRMpic)




Deleting RM_： richmenu-c1a77ada79f2364191337673d07752f4
Deleting RM_： richmenu-5caa9968280b4648dc2d54cdf57e1a4a
Deleting RM_： richmenu-86a5deeaa8574bbefe637626f0821692
Deleting RM_： richmenu-2e1d8ec8c61289c995566c5dd547e5f9
[]


In [67]:
######################## responses ########################
# 10. 入口應答：使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # get request body as text
    body = request.get_data(as_text=True)
    # 記錄用戶log
    f = open(apppath_eventlog, "a")
    f.write(body)
    f.close()
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

In [68]:
################### postback_data_map ##################
def do_goSTART(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(
            text='(幾分鐘後...)',
            quick_reply=QuickReply(
                [
                    QuickReplyButton(
                        action=PostbackTemplateAction(
                            label='確定',
                            data='startframe',
                        )
                    ),
                ]
            )
        ),
    )
#----------------------- 事務所外 ----------------------
# 每輪起始
def do_startframe(event):
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.link_rich_menu_to_user(event.source.user_id,OutdoorRMid)
    line_bot_api.reply_message(
        event.reply_token,
        startframe_Messages
    )
# 開頭
def do_opening(event):
    line_bot_api.reply_message(
        event.reply_token,
        [
            TextSendMessage(text='你在路上撿到某人名片：'),
            bussinesscardICTM()
        ]
    )
# 看看佈告
def do_lookPosts(event):
    line_bot_api.reply_message(
        event.reply_token,
        [
            TextSendMessage(text='佈告欄上什麼也沒有\n'),
            TextSendMessage(
                sender=senderFortuneTeller,
                text='(其實是還沒想到要放什麼)'),
            TextSendMessage(
                sender=senderMainActor,
                text='(未來若有完整故事線\n大概就會從這裡觸發)'),
        ]
    )
# 占卜攤
def do_fortuneTeller(event):
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.link_rich_menu_to_user(event.source.user_id,ftRMid)
    line_bot_api.reply_message(
        event.reply_token,
        ftask_Messages
    )    
# 往樓上事務所
def do_walkinOffice(event):
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.link_rich_menu_to_user(event.source.user_id,OfficeRMid)
    line_bot_api.reply_message(
        event.reply_token,
        firstask_Messages
    )
#----------------------- 事務所內 ----------------------
# 占卜介紹
def do_introduction_FT(event):
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              sender=senderFortuneTeller,
              text=''+
                  ' 【占卜攤】'+\
                  '\n'+\
                  '	抽一張卡，'+\
                  '\n'+\
                  '	或者連抽五張，'+\
                  '\n'+\
                  '	讓塔羅牌為你揭示命運。'+\
                  '\n'+\
                  '	(目前暫不提供解牌服務)',
              quick_reply=QuickReply(
                  items = [
                      QuickReplyButton(
                          action=PostbackTemplateAction(
                              label='占卜師卻不提供解牌？',
                              text='不會解牌算什麼占卜師啊...',
                              data='questYourProfessionality_FT1'
                          )
                      ),
                  ]
              )
          ),
        ]
    )
# 質疑專業_占卜
def do_questYourProfessionality_FT1(event):
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              text='......',
          ),
          TextSendMessage(
              sender=senderFortuneTeller,
              text='......',
              quick_reply=QuickReply(
                  items = [
                      QuickReplyButton(
                          action=PostbackTemplateAction(
                              label='......',
                              data='questYourProfessionality_FT2'
                          )
                      ),
                  ]
              )
          ),
        ]
    )
def do_questYourProfessionality_FT2(event):
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              text='...啊我就來不及寫啊'
          ),
        ]
    )
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.link_rich_menu_to_user(event.source.user_id,ftRMid)
# 塔羅抽牌
def do_functionTarot(event,others_):
    intN=int(others_[0])
    if 'Ready' not in others_:
        line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    sender=senderFortuneTeller,
                    text='心中默想問題，直面命運...',
                    quick_reply=QuickReply(
                        items = [
                            QuickReplyButton(
                                action=PostbackTemplateAction(
                                    label='(抽牌)',
                                    data=f'functionTarot&{intN}&Ready'
                                )
                            ),
                        ]
                    )
                ),
            ]
        )
    else:
        result_=random.sample(TarotCards,intN)
        result_={ card_:TarotDetails[card_] for card_ in result_ }
        line_bot_api.reply_message(
            event.reply_token,
            [
                tarotCTM(result_,intN)
            ]
        )
#----------------------- 事務所內 ----------------------
# 聽聽介紹
def do_introduction(event):
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              sender=senderMainActor,
              text=''+
                  '我是致力於探究「選擇」的名偵探──泫則睏男。'+
                  '\n'+
                  '本所的主要業務如下：'+
                  '\n\n'+
                  '1. 【送禮建議】'+
                  '\n'+
                  '	透過提問引導，'+
                  '\n'+
                  '	為對象挑選合適禮物。'+
                  '\n'+
                  '2. 【電腦選購】'+
                  '\n'+
                  '	透過提問引導，'+
                  '\n'+
                  '	為您篩選出適當的選擇。'+
                  '\n'+
                  '3. 【宇宙機率空間取樣法】'+
                  '\n'+
                  '	一切就交給命運安排吧。'+
                  '\n\n'+
                  '打上\'@重開\'可重來'+
                  '\n'+
                  '打上\'@回報\'可去信開發者'+
                  '\n'+
                  ' (例如：@回報 我不叫愉悅)',
          ),
        ]
    )
# 送禮建議
def do_functionGiftAdv(event):
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              text='\n-------------------\n這功能等待招商中\n(絕不是懶得爬網站)'
          ),
          TextSendMessage(
              sender=senderMainActor,
              text='順便能進行市場調查喔'
          ),
        ]
    )
# 電腦選購
def do_functionBuyComputer(event):
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              text='\n-------------------\n這功能等待招商中\n(絕不是懶得設計選項)'
          ),
          TextSendMessage(
              sender=senderFortuneTeller,
              text='藉推薦之名行推銷之實'
          ),
        ]
    )
# # 等等吃啥
# def do_functionEatWhat(event):
#     line_bot_api.reply_message(
#         event.reply_token,
#         [
#             TextSendMessage(
#                 sender=senderMainActor,
#                 text='請點擊"分享位置"按鈕分享定位',
#                 quick_reply=produceQuickReplyItem(QRB_Location("分享位置","https://i.imgur.com/wN8eSzR.jpeg"))
#             ),
#         ]
#     )
# 宇宙機率空間取樣法
def do_functionDice(event):
    user__user_id=event.source.user_id
    with open(apppath_useroperatestatecsv, "r") as csvfile_:
        csv_data=list(csv.reader(csvfile_))
    for i_ in range(len(csv_data)):
        if csv_data[i_][0]==user__user_id:
            csv_data[i_][1]=''
            with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                writer_=csv.writer(csvfile_)
                writer_.writerows(csv_data)
            break
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.link_rich_menu_to_user(event.source.user_id,diceRMid)
    try:
        line_bot_api.reply_message(
            event.reply_token,
            [
              TextSendMessage(
                  text='(------ 我是分隔線 ------)',
              ),
              TextSendMessage(
                  sender=senderMainActor,
                  text='想要來賭...\n我是說，需要我的專業嗎？',
              ),
            ]
        )
    except:
      return
#----------------------- 骰子選單 ----------------------
# 骰子介紹
def do_introduction_Dice(event):
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              sender=senderMainActor,
              text=''+
                  '【宇宙機率空間取樣法】'+\
                  '\n'+\
                  '將命運交給大宇宙的隨機性。'+\
                  '\n'+\
                  '1. 『自訂骰子』'+\
                  '\n'+\
                  '	查看自訂骰子。'+\
                  '\n'+\
                  '2. 『俄羅斯輪盤』'+\
                  '\n'+\
                  '	輸入啞彈與實彈數量，'+\
                  '\n'+\
                  '	直至實彈打光為止。'+\
                  '\n'+\
                  '	(可上傳自訂的實彈命中圖)'+\
                  '\n'+\
                  '3. 『經典亂數骰』'+\
                  '\n'+\
                  '	輸入骰子面數，'+\
                  '\n'+\
                  '	然後骰起來。'+\
                  '\n'+\
                  '	(可自訂骰面所顯示的項目)',
              quick_reply=QuickReply(
                  items = [
                      QuickReplyButton(
                          action=PostbackTemplateAction(
                              label='這跟偵探有什麼關係？',
                              text='不就單純的賭運氣和擲骰子嘛',
                              data='questYourProfessionality1'
                          )
                      ),
                  ]
              )
          ),
        ]
    )
# 質疑專業
def do_questYourProfessionality1(event):
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              sender=senderMainActor,
              text='......',
          ),
          TextSendMessage(
              text='......',
              quick_reply=QuickReply(
                  items = [
                      QuickReplyButton(
                          action=PostbackTemplateAction(
                              label='......',
                              data='questYourProfessionality2'
                          )
                      ),
                  ]
              )
          ),
        ]
    )
def do_questYourProfessionality2(event):
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              sender=senderMainActor,
              text='...請尊重專業。'
          ),
        ]
    )
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.link_rich_menu_to_user(event.source.user_id,diceRMid)
# 自訂骰子一覽
def do_functionDiceList(event):
    temp__if_user_not_exist=True
    text_bullet="\n你尚未自訂實彈命中圖片。"
    text_dice="\n你尚未自訂骰面顯示項目。"
    user__user_id=event.source.user_id
    user__cus_bullet=''
    user__cus_dice=''
    with open(apppath_useroperatestatecsv,mode='r') as csvfile_:
        csvDictReader=csv.DictReader(csvfile_)
        for row in csvDictReader:
            if row['usr_id']==user__user_id:
                temp__if_user_not_exist=False
                if row['cus_bullet']!='':
                    user__cus_bullet=row['cus_bullet']
                    global ngrok_url
                    text_bullet="你已自訂的實彈命中圖片:"\
                          +'\n'\
                          +f"{user__cus_bullet}"
                if row['cus_dice']!='':
                    user__cus_dice=row['cus_dice']
                    text_dice="你已自訂的骰面:"
                    list_=user__cus_dice.split('^$^')
                    i_=0
                    for item_ in list_:
                      i_+=1
                      text_dice+=f'\n{i_}.{item_}'
                break
    if temp__if_user_not_exist:
        with open(apppath_useroperatestatecsv,mode='a') as csvfile_:
            writer_ = csv.writer(csvfile_)
            writer_.writerow([user__user_id,'','','',''])
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              sender=senderMainActor,
              text=''
                +text_bullet
                +'\n\n'
                +text_dice,
          ),
        ]
    )
# 俄羅斯輪盤=================
def do_functionBasketBet(event):
    temp__if_user_not_exist=True
    text_="你尚未自訂實彈命中圖片，"\
        +'\n'\
        +"是否新增或使用預設圖？"
    user__user_id=event.source.user_id
    user__cus_bullet=''
    QRBItems_=[]
    QRBItems_.append(
        QuickReplyButton(
            action=PostbackTemplateAction(
                label='新自訂圖片',
                data=f'cusBullet&user_id={user__user_id}'
            )
        ),
    )
    with open(apppath_useroperatestatecsv,mode='r') as csvfile_:
        csvDictReader=csv.DictReader(csvfile_)
        for row in csvDictReader:
            if row['usr_id']==user__user_id:
                temp__if_user_not_exist=False
                if row['cus_bullet']!='':
                    user__cus_bullet=row['cus_bullet']
                    user__cus_bullet_audio=row['cus_bullet_audio']
                    text_="你有自訂的實彈命中圖，"\
                        +'\n'\
                        +"重新自訂、使用已自訂圖片、"\
                        +'\n'\
                        +"或者使用預設命中圖片？"
                    QRBItems_.append(
                        QuickReplyButton(
                            action=PostbackTemplateAction(
                                label='使用自訂圖片',
                                data=f'useCusBullet&user_id={user__user_id}&cus_bullet={user__cus_bullet}^$^{user__cus_bullet_audio}'
                            )
                        ),
                    )
                break
    if temp__if_user_not_exist:
        with open(apppath_useroperatestatecsv,mode='a') as csvfile_:
            writer_ = csv.writer(csvfile_)
            writer_.writerow([user__user_id,'','','',''])
    QRBItems_.append(
        QuickReplyButton(
            action=PostbackTemplateAction(
                label='使用預設圖片',
                data=f'useDefBullet&user_id={user__user_id}'
            )
        ),
    )
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.reply_message(
        event.reply_token,
        [
            TextSendMessage(
                sender=senderMainActor,
                text=text_,
                quick_reply=QuickReply(
                    QRBItems_
                )
            ),
        ]
    )
# 新自訂命中圖片
def do_cusBullet(event,others_):
    user__user_id=others_[0].split('=')[1]
    with open(apppath_useroperatestatecsv, "r") as csvfile_:
        csv_data=list(csv.reader(csvfile_))
    for i_ in range(len(csv_data)):
        if csv_data[i_][0]==user__user_id:
            print(csv_data[i_][1])
            csv_data[i_][1]=f'cusBullet&user_id={user__user_id}'
            print("->",csv_data[i_][1])
            with open(apppath_useroperatestatecsv,"w",newline='') as csvfile_:
                writer_=csv.writer(csvfile_)
                writer_.writerows(csv_data)
            break
    line_bot_api.reply_message(
        event.reply_token,
        [
            TextSendMessage(
                sender=senderMainActor,
                text="點擊按鈕選擇並上傳自訂圖片：",
                quick_reply=QuickReply(
                    [
                        QuickReplyButton(
                            action=CameraRollAction(
                                label='上傳自訂圖片：',
                            )
                        ), 
                    ]
                )
            ),
        ]
    )
def do_giveupAudio(event):
    user__user_id=event.source.user_id
    with open(apppath_useroperatestatecsv, "r") as csvfile_:
        csv_data=list(csv.reader(csvfile_))
    for i_ in range(len(csv_data)):
        if csv_data[i_][0]==user__user_id:
            csv_data[i_][4]=''
            with open(apppath_useroperatestatecsv,"w",newline='') as csvfile_:
                writer_=csv.writer(csvfile_)
                writer_.writerows(csv_data)
            break
    do_functionDice(event)
# 使用自訂命中圖片
def do_useCusBullet(event,others_):
    user__user_id=others_[0].split('=')[1]
    user__cus_bullet_=others_[1].split('=')[1]
    user__cus_bullet=user__cus_bullet_.split('^$^')[0]
    user__cus_bullet_audio=user__cus_bullet_.split('^$^')[1]
    if "Ready" in others_:
        trueBulletN=int(others_[2])
        if trueBulletN<=0:
            line_bot_api.reply_message(
                event.reply_token,
                [
                    TextSendMessage(
                        sender=senderMainActor,
                        text='實彈已全數打完',
                    ),
                ]
            )
            do_functionDice(event)
            return
        falseBulletN=int(others_[3])
        clip_=[True]*trueBulletN+[False]*falseBulletN
        if clip_[GoDice(trueBulletN+falseBulletN)]:
            trueBulletN-=1
            temp_bangtext='Bang！ (下一位)'
            if user__cus_bullet_audio!='':
            # 語音轉文字ˇˇˇˇˇˇˇˇˇˇˇˇˇˇˇˇˇˇˇ
                sr_ = sr.Recognizer()
                r_=requests.get(user__cus_bullet_audio)
                with open('temp_', 'wb') as file:
                    file.write(r_.content)
                sound = AudioSegment.from_file("temp_")
                sound.export("temp_.wav", format="wav")
                !rm temp_
                with sr.AudioFile('temp_.wav') as source:
                    audio = sr_.record(source)
                !rm temp_.wav
                cc = OpenCC('s2twp')
                StoT_=cc.convert(sr_.recognize_google(audio,language='zh-TW'))
                print("#@$!%",StoT_)
            # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                temp_bangtext=f"\"{StoT_}\""
            messages_=[
                TemplateSendMessage(
                    alt_text='Image Carousel Template',
                    template=ImageCarouselTemplate(
                        columns=
                        [
                            ImageCarouselColumn(
                                image_url=f"{user__cus_bullet}",
                                action=PostbackTemplateAction(
                                    label=temp_bangtext,
                                    data=f'useCusBullet&user_id={user__user_id}&cus_bullet={user__cus_bullet}^$^{user__cus_bullet_audio}&{trueBulletN}&{falseBulletN}&Ready'
                                )
                            )
                        ]
                    )
                ),
            ]
            if user__cus_bullet_audio!='':
                messages_.append(
                    AudioSendMessage(
                        original_content_url=f"{user__cus_bullet_audio}",
                        duration=3
                    )
                )
            line_bot_api.reply_message(
                event.reply_token,
                messages_
            )
        else:
            falseBulletN-=1
            line_bot_api.reply_message(
                event.reply_token,
                [
                    TemplateSendMessage(
                        alt_text='Image Carousel Template',
                        template=ImageCarouselTemplate(
                            columns=
                            [
                                ImageCarouselColumn(
                                    image_url=imgurl_Miss,
                                    action=PostbackTemplateAction(
                                        label='Miss~ (下一位)',
                                        data=f'useCusBullet&user_id={user__user_id}&cus_bullet={user__cus_bullet}^$^{user__cus_bullet_audio}&{trueBulletN}&{falseBulletN}&Ready'
                                    )
                                )
                            ]
                        )
                    ),
                ]
            )
    elif len(others_)>=4:
        trueBulletN=int(others_[2])
        falseBulletN=int(others_[3])
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    sender=senderMainActor,
                    text='準備好就開第一槍吧',
                    quick_reply=QuickReply(
                        [
                            QuickReplyButton(
                                action=PostbackTemplateAction(
                                    label='開第一槍！',
                                    data=f'useCusBullet&user_id={user__user_id}&cus_bullet={user__cus_bullet}^$^{user__cus_bullet_audio}&{trueBulletN}&{falseBulletN}&Ready'
                                )
                            ),
                        ]
                    )
                ),
            ]
        )
    elif len(others_)>=3:
        trueBulletN=int(others_[2])
        with open(apppath_useroperatestatecsv, "r") as csvfile_:
            csv_data=list(csv.reader(csvfile_))
        for i_ in range(len(csv_data)):
            if csv_data[i_][0]==user__user_id:
                csv_data[i_][1]=f'useCusBullet&user_id={user__user_id}&cus_bullet={user__cus_bullet}^$^{user__cus_bullet_audio}&{trueBulletN}'
                with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                    writer_=csv.writer(csvfile_)
                    writer_.writerows(csv_data)
                break
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    sender=senderMainActor,
                    text='要裝入的啞彈數量：',
                ),
            ]
        )
    else:
        with open(apppath_useroperatestatecsv, "r") as csvfile_:
            csv_data=list(csv.reader(csvfile_))
        for i_ in range(len(csv_data)):
            print(csv_data[i_])
            if csv_data[i_][0]==user__user_id:
                csv_data[i_][1]=f'useCusBullet&user_id={user__user_id}&cus_bullet={user__cus_bullet}^$^{user__cus_bullet_audio}'
                with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                    writer_=csv.writer(csvfile_)
                    writer_.writerows(csv_data)
                print("->",csv_data[i_])
                break
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    sender=senderMainActor,
                    text='要裝入的實彈數量：',
                ),
            ]
        )
# 使用預設命中圖片
def do_useDefBullet(event,others_):
    user__user_id=others_[0].split('=')[1]
    if "Ready" in others_:
        print("Ready",others_)
        trueBulletN=int(others_[1])
        if trueBulletN<=0:
            line_bot_api.reply_message(
                event.reply_token,
                [
                    TextSendMessage(
                        sender=senderMainActor,
                        text='實彈已全數打完',
                    ),
                ]
            )
            do_functionDice(event)
            return
        falseBulletN=int(others_[2])
        clip_=[True]*trueBulletN+[False]*falseBulletN
        if clip_[GoDice(trueBulletN+falseBulletN)]:
            trueBulletN-=1
            line_bot_api.reply_message(
                event.reply_token,
                [
                    TemplateSendMessage(
                        alt_text='Image Carousel Template',
                        template=ImageCarouselTemplate(
                            columns=
                            [
                                ImageCarouselColumn(
                                    image_url=imgurl_Bang,
                                    action=PostbackTemplateAction(
                                        label='Bang！ (下一位)',
                                        data=f'useDefBullet&user_id={user__user_id}&{trueBulletN}&{falseBulletN}&Ready'
                                    )
                                )
                            ]
                        )
                    ),
                ]
            )
        else:
            falseBulletN-=1
            line_bot_api.reply_message(
                event.reply_token,
                [
                    TemplateSendMessage(
                        alt_text='Image Carousel Template',
                        template=ImageCarouselTemplate(
                            columns=
                            [
                                ImageCarouselColumn(
                                    image_url=imgurl_Miss,
                                    action=PostbackTemplateAction(
                                        label='Miss~ (下一位)',
                                        data=f'useDefBullet&user_id={user__user_id}&{trueBulletN}&{falseBulletN}&Ready'
                                    )
                                )
                            ]
                        )
                    ),
                ]
            )
    elif len(others_)>=3:
        trueBulletN=int(others_[1])
        falseBulletN=int(others_[2])
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    sender=senderMainActor,
                    text='準備好就開第一槍吧',
                    quick_reply=QuickReply(
                        [
                            QuickReplyButton(
                                action=PostbackTemplateAction(
                                    label='開第一槍！',
                                    data=f'useDefBullet&user_id={user__user_id}&{trueBulletN}&{falseBulletN}&Ready'
                                )
                            ),
                        ]
                    )
                ),
            ]
        )
    elif len(others_)>=2:
        trueBulletN=int(others_[1])
        with open(apppath_useroperatestatecsv, "r") as csvfile_:
            csv_data=list(csv.reader(csvfile_))
        for i_ in range(len(csv_data)):
            if csv_data[i_][0]==user__user_id:
                csv_data[i_][1]=f'useDefBullet&user_id={user__user_id}&{trueBulletN}'
                with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                    writer_=csv.writer(csvfile_)
                    writer_.writerows(csv_data)
                break
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    sender=senderMainActor,
                    text='要裝入的啞彈數量：',
                ),
            ]
        )
    else:
        with open(apppath_useroperatestatecsv, "r") as csvfile_:
            csv_data=list(csv.reader(csvfile_))
        for i_ in range(len(csv_data)):
            print(csv_data[i_])
            if csv_data[i_][0]==user__user_id:
                csv_data[i_][1]=f'useDefBullet&user_id={user__user_id}'
                with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                    writer_=csv.writer(csvfile_)
                    writer_.writerows(csv_data)
                print("->",csv_data[i_])
                break
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    sender=senderMainActor,
                    text='要裝入的實彈數量：',
                ),
            ]
        )
# 經典亂數骰=================
def do_functionNormalDice(event):
    temp__if_user_not_exist=True
    text_="你尚未自訂骰面顯示項目，"\
        +'\n'\
        +"是否新增或使用預設數字骰面？"
    user__user_id=event.source.user_id
    user__cus_dice=''
    QRBItems_=[]
    QRBItems_.append(
        QuickReplyButton(
            action=PostbackTemplateAction(
                label='新自訂骰面',
                data=f'cusDice&user_id={user__user_id}'
            )
        ),
    )
    with open(apppath_useroperatestatecsv,mode='r') as csvfile_:
        csvDictReader=csv.DictReader(csvfile_)
        for row in csvDictReader:
            if row['usr_id']==user__user_id:
                temp__if_user_not_exist=False
                if row['cus_dice']!='':
                  user__cus_dice=row['cus_dice']
                  text_="你有自訂的骰面顯示項目，"\
                      +'\n'\
                      +"重新自訂、使用已自訂骰面、"\
                      +'\n'\
                      +"或者使用預設數字骰面？"
                  QRBItems_.append(
                      QuickReplyButton(
                          action=PostbackTemplateAction(
                              label='使用自訂骰面',
                              data=f'useCusDice&user_id={user__user_id}&cus_dice={user__cus_dice}'
                          )
                      ),
                  )
                break
    if temp__if_user_not_exist:
        with open(apppath_useroperatestatecsv,mode='a') as csvfile_:
            writer_ = csv.writer(csvfile_)
            writer_.writerow([user__user_id,'','','',''])
    QRBItems_.append(
        QuickReplyButton(
            action=PostbackTemplateAction(
                label='使用預設骰面',
                data=f'useDefDice&user_id={user__user_id}'
            )
        ),
    )
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.reply_message(
        event.reply_token,
        [
          TextSendMessage(
              sender=senderMainActor,
              text=text_,
              quick_reply=QuickReply(
                  QRBItems_
              )
          ),
        ]
    )
# 新自訂骰面
def do_cusDice(event,others_):
    user__user_id=others_[0].split('=')[1]
    with open(apppath_useroperatestatecsv, "r") as csvfile_:
        csv_data=list(csv.reader(csvfile_))
    for i_ in range(len(csv_data)):
        if csv_data[i_][0]==user__user_id:
            if len(others_)>1:
                csv_data[i_][3]='^$^'.join(others_[1:])
                with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                    writer_=csv.writer(csvfile_)
                    writer_.writerows(csv_data)
                line_bot_api.reply_message(
                    event.reply_token,
                    [
                      TextSendMessage(
                          text="(自訂骰面保存成功)",
                      )
                    ]
                )
                do_functionDice(event)
            else:
                csv_data[i_][1]=f'cusDice&user_id={user__user_id}'
                with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                    writer_=csv.writer(csvfile_)
                    writer_.writerows(csv_data)
                line_bot_api.reply_message(
                    event.reply_token,
                    [
                      TextSendMessage(
                          sender=senderMainActor,
                          text="將骰面一項一項輸入，\n完畢後點擊「輸入完成」：\n(或隨時點擊「取消」)",
                          quick_reply=QuickReply(
                              [
                                  QuickReplyButton(
                                      action=PostbackTemplateAction(
                                          label='輸入完成',
                                          data=f'cusDice&user_id={user__user_id}'
                                      )
                                  ),
                                  QuickReplyButton(
                                      action=PostbackTemplateAction(
                                          label='取消',
                                          data='functionDice'
                                      )
                                  ),
                              ]
                          )
                      ),
                    ]
                )
            break
# 使用自訂骰面
def do_useCusDice(event,others_):
    user__user_id=others_[0].split('=')[1]
    user__cus_dice=others_[1].split('=')[1].split('^$^')
    theOne_=user__cus_dice[GoDice(len(user__cus_dice))]
    text_=''
    if 'RE' in others_:
        text_=f"就決定是你了，\n【{theOne_}】！"
    else:
        text_=f"......\n.....\n....\n...\n\n就決定是你了，\n【{theOne_}】！"
    line_bot_api.reply_message(
        event.reply_token,
        [
            TextSendMessage(
                sender=senderMainActor,
                text=text_,
                quick_reply=QuickReply(
                    [
                        QuickReplyButton(
                            action=PostbackTemplateAction(
                                label='再來一次',
                                data=f'useCusDice&user_id={user__user_id}&{others_[1]}&RE'
                            )
                        ),
                        QuickReplyButton(
                            action=PostbackTemplateAction(
                                label='結束',
                                data='functionDice'
                            )
                        ),
                    ]
                )
            ),
        ]
    )
# 使用預設骰面
def do_useDefDice(event,others_):
    user__user_id=others_[0].split('=')[1]
    intN=0
    if len(others_)>1:
        intN=int(others_[1])
        theOne_=GoDice(intN)+1
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    sender=senderMainActor,
                    text=f"結果是： {theOne_}",
                    quick_reply=QuickReply(
                        [
                            QuickReplyButton(
                                action=PostbackTemplateAction(
                                    label='再來一次',
                                    data=f'useDefDice&user_id={user__user_id}&{intN}'
                                )
                            ),
                            QuickReplyButton(
                                action=PostbackTemplateAction(
                                    label='結束',
                                    data='functionDice'
                                )
                            ),
                        ]
                    )
                ),
            ]
        )
    else:
        with open(apppath_useroperatestatecsv, "r") as csvfile_:
            csv_data=list(csv.reader(csvfile_))
        for i_ in range(len(csv_data)):
            print(csv_data[i_])
            if csv_data[i_][0]==user__user_id:
                csv_data[i_][1]=f'useDefDice&user_id={user__user_id}'
                with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                    writer_=csv.writer(csvfile_)
                    writer_.writerows(csv_data)
                print("->",csv_data[i_])
                break
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    sender=senderMainActor,
                    text="請輸入骰子面數：",
                ),
            ]
        )
def GoDice(intN):
    return random.randint(0,intN-1)
#----------------------- data_map ----------------------
postback_data_map = {
    # 開頭
    "opening": do_opening,
    # 每輪起始
    "goSTART":do_goSTART,
    "startframe":do_startframe,
    #- - - - 事務所外 - - - - - - - - -
    # 看看佈告
    "lookPosts":do_lookPosts,
    # 占卜攤
    "fortuneTeller":do_fortuneTeller,
    # 往樓上事務所
    "walkinOffice":do_walkinOffice,
    #- - - - 占卜選單 - - - - - - - - -
    # 占卜介紹
    "introduction_FT":do_introduction_FT,
        # 質疑專業
        "questYourProfessionality_FT1":do_questYourProfessionality_FT1,
        "questYourProfessionality_FT2":do_questYourProfessionality_FT2,
    # 塔羅抽牌
    "functionTarot":do_functionTarot,
    "FTspeak":do_FTspeak,
    #- - - - 事務所內 - - - - - - - - -
    # 聽聽介紹
    "introduction":do_introduction,
    # 電腦選購
    "functionBuyComputer":do_functionBuyComputer,
    # 送禮建議
    "functionGiftAdv":do_functionGiftAdv,
    # # 等等吃啥
    # "functionDice":do_functionDice,
    # 宇宙機率空間多面單一取樣法
    "functionDice":do_functionDice,
    #- - - - 骰子選單 - - - - - - - - -
    # 骰子介紹
    "introduction_Dice":do_introduction_Dice,
        # 質疑專業
        "questYourProfessionality1":do_questYourProfessionality1,
        "questYourProfessionality2":do_questYourProfessionality2,
    # 自訂骰子一覽
    "functionDiceList":do_functionDiceList,
    # 俄羅斯輪盤骰
    "functionBasketBet":do_functionBasketBet,
        # 自訂命中圖片
        "cusBullet":do_cusBullet,
        "giveupAudio":do_giveupAudio,
        # 使用自訂命中圖片
        "useCusBullet":do_useCusBullet,
        # 使用預設命中圖片
        "useDefBullet":do_useDefBullet,
    # 經典亂數骰
    "functionNormalDice":do_functionNormalDice,
        # 新自訂骰面
        "cusDice":do_cusDice,
        # 使用自訂骰面
        "useCusDice":do_useCusDice,
        # 使用預設骰面
        "useDefDice":do_useDefDice,
    #- - - - - - - - - - - - - - - -
    # 離開事務所
    "walkoutOffice":do_startframe,
}

In [69]:
##################### events handler ######################
# 10. 入口訊息：This is "TEST0"
@app.route("/", methods=['GET'])
def welcome():
    ngrok_url=f"https://{dict(request.headers)['Host']}"
    print(ngrok_url)
    return f" - This is \"{appname}\" - "
# 10. handler添加：如果收到FollowEvent，則記錄用戶資料
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)   
    # 將用戶資訊存在檔案內
    with open(apppath_userlog, "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    # 回覆文字消息與圖片消息
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
    line_bot_api.reply_message(
        event.reply_token,
        [
            openingICTM()
        ]
    )
# 10. handler添加：如果收到UnfollowEvent，則unlink圖文選單
@handler.add(UnfollowEvent)
def unFollow(event):
    line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
# # 10. handler添加：如果收到位置訊息
# @handler.add(MessageEvent, message=LocationMessage)
# def handle_message(event):
#     print(event.message)
#     line_bot_api.reply_message(
#         event.reply_token,
#         [
#             TextSendMessage(
#                 text='這功能沒法做了\n(不想花錢做)'
#             ),
#             TextSendMessage(
#                 sender=senderMainActor,
#                 text='GoogleMapAPI要錢了...\n嗚嗚嗚...'
#             ),
#         ]
#     )
# 10. handler添加：如果收到圖片訊息
@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    print("ImageMessage")
    user__user_id=event.source.user_id
    # 這部分原本要用imgur api做，但這陣子由於imgur本身的問題Access Token申請不了，嗚嗚嗚
    with open(apppath_useroperatestatecsv, "r") as csvfile_:
        csv_data=list(csv.reader(csvfile_))
    for i_ in range(len(csv_data)):
        if csv_data[i_][0]==user__user_id:
            if csv_data[i_][1].split('&')[0]=='cusBullet':
                message_content=line_bot_api.get_message_content(event.message.id)
                storefile_=f"{event.source.user_id}.jpeg"
                with open(storefile_,"wb") as f:
                    for chunk in message_content.iter_content():
                        f.write(chunk)
                bucket_name='sheng123456'
                bucket = storage_client.bucket(bucket_name)
                blob=bucket.blob(storefile_)
                #blob.delete()
                blob.upload_from_filename(storefile_)
                # !rm storefile_
                #blob.make_public()
                imgpath_url= blob.public_url
                print(f'upload image as: {imgpath_url}')
                csv_data[i_][2]=f'{imgpath_url}'
                with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                    writer_=csv.writer(csvfile_)
                    writer_.writerows(csv_data)
                line_bot_api.reply_message(
                    event.reply_token,
                    [
                        TextSendMessage(
                            sender=senderMainActor,
                            text='可追加(3秒10字以內)錄音：',
                            quick_reply=QuickReply(
                                [
                                    QuickReplyButton(
                                        action=PostbackTemplateAction(
                                            label='放棄',
                                            text='(自訂圖片保存成功)',
                                            data='giveupAudio'
                                        )
                                    ),
                                ]
                            )
                        ),
                    ]
                )
                #do_functionDice(event)
            break
# 10. handler添加：如果收到聲音訊息
@handler.add(MessageEvent, message=AudioMessage)
def handle_audio_message(event):
    print("AudioMessage")
    user__user_id=event.source.user_id
    with open(apppath_useroperatestatecsv, "r") as csvfile_:
        csv_data=list(csv.reader(csvfile_))
    for i_ in range(len(csv_data)):
        if csv_data[i_][0]==user__user_id:
            if csv_data[i_][1].split('&')[0]=='cusBullet':
                message_content=line_bot_api.get_message_content(event.message.id)
                storefile_=f"{event.source.user_id}.m4a"
                with open(storefile_,"wb") as f:
                    for chunk in message_content.iter_content():
                        f.write(chunk)
                bucket_name='sheng123456'
                bucket = storage_client.bucket(bucket_name)
                blob=bucket.blob(storefile_)
                #blob.delete()
                blob.upload_from_filename(storefile_)
                # blobs = bucket.list_blobs(prefix='some/directory')
                # for blob_ in blobs:
                #   print("> ",blob_.public_url)
                # !rm storefile_
                #blob.make_public()
                audiopath_url= blob.public_url
                print(f'upload audio as: {audiopath_url}')
                csv_data[i_][4]=f'{audiopath_url}'
                with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                    writer_=csv.writer(csvfile_)
                    writer_.writerows(csv_data)
                line_bot_api.reply_message(
                    event.reply_token,
                    [
                        TextSendMessage(
                            sender=senderMainActor,
                            text='(自訂圖片&追加錄音成功)',
                        ),
                    ]
                )
                do_functionDice(event)
            break
# 10. handler添加：如果收到文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    print(event.message.text)
    if event.message.text.startswith('@回報'):
        with open(apppath_bugreportlog, "a") as myfile:
            myfile.write(f'{event.source.user_id}：\t{event.message.text}\n')
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    text='收到囉，感謝來信。'
                ),
            ]
        )
        return
    elif event.message.text.startswith('@重開'):
        line_bot_api.unlink_rich_menu_from_user(event.source.user_id)
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(
                    text='你回想起一切的開端...'
                ),
                openingICTM()
            ]
        )
        return
    user__user_id=event.source.user_id
    try:
        intN=int(event.message.text)
        with open(apppath_useroperatestatecsv,mode='r') as csvfile_:
            csvDictReader=csv.DictReader(csvfile_)
            for row in csvDictReader:
                if row['usr_id']==user__user_id:
                    print(row)
                    if row['operate_state']!='':
                        data_=row['operate_state'].split('&')
                        data_.append(intN)
                        if len(data_)>1:
                            postback_data_map[data_[0]](event,data_[1:])
                        else:
                            postback_data_map[data_[0]](event)
                        break
                        print(f"[{data_[0]}]")
    except:
        user__user_id=event.source.user_id
        with open(apppath_useroperatestatecsv, "r") as csvfile_:
            csv_data=list(csv.reader(csvfile_))
        for i_ in range(len(csv_data)):
            if csv_data[i_][0]==user__user_id:
                if csv_data[i_][1].split('&')[0]=='cusDice':
                    csv_data[i_][1]+=f'&{event.message.text}'
                    with open(apppath_useroperatestatecsv, "w",newline='') as csvfile_:
                        writer_=csv.writer(csvfile_)
                        writer_.writerows(csv_data)
                    line_bot_api.reply_message(
                        event.reply_token,
                        [
                            TextSendMessage(
                                sender=senderMainActor,
                                text='下一項：',
                                quick_reply=QuickReply(
                                    [
                                        QuickReplyButton(
                                            action=PostbackTemplateAction(
                                                label='輸入完成',
                                                data=f'{csv_data[i_][1]}'
                                            )
                                        ),
                                        QuickReplyButton(
                                            action=PostbackTemplateAction(
                                                label='取消',
                                                data='functionDice'
                                            )
                                        ),
                                    ]
                                )
                            )
                        ]
                    )
                break
# 10. handler添加：如果收到PostbackEvent，做出相應處理
@handler.add(PostbackEvent)
def handle_post_message(event):
    print(event.postback.data)
    data_=event.postback.data.split('&')
    m_=data_[0]
    if m_ in postback_data_map:
      if len(data_)<=1:
        postback_data_map[m_](event)
      else:
        postback_data_map[m_](event,data_[1:])

      

In [ ]:
######################### run ############################
# 11. 啟動主程序
app.run()